# Denison CS181/DA210 Homework 4.d - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds(source="sqlite2")
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/school.db


#### Establish Connection from Client to Server

In [4]:
%sql $cstring

---

## Part A: Advanced Partitioning and Grouping

**Q1:** In reference to the `school` database, list, for each course, the number of class sections being taught in the spring.  Call your new column `num_sections`.

In [5]:
query = """SELECT coursesubject, coursenum, COUNT (classsection) as num_sections 
FROM classes
WHERE classterm == 'SPRING'
GROUP BY coursesubject, coursenum
"""

resultset = %sql $query
resultdf1 = resultset.DataFrame()
resultdf1

 * sqlite:///../../dbfiles/school.db
Done.


,coursesubject,coursenum,num_sections
0,ARAB,112,1
1,ARTH,157,1
2,ARTH,251,1
3,ARTH,258,1
4,ARTH,263,3
...,...,...,...
508,WMST,307,1
509,WMST,345,2
510,WMST,362,3
511,WMST,390,1


In [6]:
# Testing cell
assert resultdf1.shape == (513, 3)
assert resultdf1.loc[0,"num_sections"] == 1
assert resultdf1.loc[4,"num_sections"] == 3

**Q2:** For each class meeting time, find the total number of students who took a class during that meeting time.  Do not include classes without a meeting time.

Call your new column `count`, and order your results by `count` (from most popular to least popular).

In [7]:
query = """SELECT cl.classmeeting, COUNT (stu.studentid) as count
FROM classes as cl, student_class as sc, students as stu
ON cl.classid == sc.classid
AND stu.studentid == sc.studentid
WHERE cl.classmeeting IS NOT NULL
GROUP BY cl.classmeeting
ORDER BY count DESC
"""


resultset = %sql $query
resultdf2 = resultset.DataFrame()
resultdf2

 * sqlite:///../../dbfiles/school.db
Done.


,classmeeting,count
0,10:00-11:20 TR,1549
1,09:30-10:20 MWF,1527
2,10:30-11:20 MWF,1361
3,11:30-12:20 MWF,1354
4,13:30-14:50 TR,1229
...,...,...
130,13:30-16:20 T,4
131,13:30-14:50 R,4
132,13:30-14:20 MTWRF,4
133,10:30-11:20 T,3


In [8]:
# Testing cell
assert len(resultdf2) == 135
assert resultdf2.loc[0,"count"] == 1549
assert resultdf2.loc[132,"count"] == 4

**Q3:** Using the `school` database, list, for each student id the total number of credits they took during the year.  Select the `studentid` column and a new column `total_credits`.

In [9]:
query = """SELECT stu.studentid, SUM (co.coursehours) as total_credits
FROM students as stu, student_class as sc, classes as cl, courses as co
ON stu.studentid == sc.studentid
AND sc.classid == cl.classid
AND cl.coursenum == co.coursenum
AND cl.coursesubject == co.coursesubject
GROUP BY stu.studentid
"""

resultset = %sql $query
resultdf3 = resultset.DataFrame()
resultdf3

 * sqlite:///../../dbfiles/school.db
Done.


,studentid,total_credits
0,61500,5.0
1,61501,3.0
2,61502,4.0
3,61503,0.0
4,61504,4.0
...,...,...
2269,63769,20.0
2270,63770,4.0
2271,63771,4.0
2272,63772,4.0


In [10]:
# Testing cell
assert len(resultdf3) == 2274
assert list(resultdf3.loc[:4,"total_credits"]) == [5.0, 3.0, 4.0, 0.0, 4.0]

---

## Part B: Subqueries

**Q4:** In reference to the `school` database, what is the average credit-hour load of the students, including directed studies?  Your answer should be a 1x1 table.

_Hint_: Use your query from Q3 as a subquery.

In [11]:
query = """SELECT AVG(total_credits) FROM
(SELECT stu.studentid, SUM (co.coursehours) as total_credits
FROM students as stu, student_class as sc, classes as cl, courses as co
ON stu.studentid == sc.studentid
AND sc.classid == cl.classid
AND cl.coursenum == co.coursenum
AND cl.coursesubject == co.coursesubject
GROUP BY stu.studentid)
"""


resultset = %sql $query
resultdf4 = resultset.DataFrame()
resultdf4.head()

 * sqlite:///../../dbfiles/school.db
Done.


,AVG(total_credits)
0,30.283641


In [12]:
# Testing cell
assert resultdf4.shape == (1,1)
assert resultdf4.iloc[0,0] > 30
assert resultdf4.iloc[0,0] < 31

**Q5:** Using the `school` database, write a SQL query to find out, for each class, how many different majors are represented among the students who took the class.  Do not count `NULL` as a major.  Order your results by `classid`, and include columns `classid`, `coursesubject`, `coursenum`, `classsection`, and a new column `num_majors`.

In [13]:
query = """SELECT cl.classid, cl.coursesubject, cl.coursenum, cl.classsection, COUNT(DISTINCT stu.studentmajor) as num_majors
FROM students as stu, student_class as sc, classes as cl, courses as co
ON stu.studentid == sc.studentid
AND sc.classid == cl.classid
AND cl.coursenum == co.coursenum
AND cl.coursesubject == co.coursesubject
WHERE stu.studentmajor IS NOT NULL
GROUP BY cl.classid
ORDER BY cl.classid
"""

resultset = %sql $query
resultdf5 = resultset.DataFrame()
print(len(resultdf5))
resultdf5.head()

 * sqlite:///../../dbfiles/school.db
Done.
1796


,classid,coursesubject,coursenum,classsection,num_majors
0,21002,INTD,75,01,6
1,21003,INTD,75,02,24
2,21004,INTD,75,03,5
3,21007,INTD,75,04,2
4,21008,INTD,75,05,4


In [14]:
# Testing cell
assert len(resultdf5) == 1796
assert list(resultdf5.columns) == ['classid', 'coursesubject', 'coursenum', 'classsection', 'num_majors']
assert resultdf5.iloc[0,4] == 6
assert resultdf5.iloc[1,4] == 24

#### Switch to the `book` database

In [15]:
scheme, dbdir, database = getsqlite_creds(source="sqlite1")
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

%sql $cstring

Connection string: sqlite:///../../dbfiles/book.db


**Q6:** Using the `book` database, which country codes appear in `indicators` but not in `countries`?

In [16]:
query = """SELECT DISTINCT code
FROM indicators
WHERE code NOT IN 
(SELECT code
FROM countries)
"""

resultset = %sql $query
resultdf6 = resultset.DataFrame()
print(len(resultdf6))
resultdf6.head()

 * sqlite:///../../dbfiles/book.db
   sqlite:///../../dbfiles/school.db
Done.
1


,code
0,INX


In [17]:
# Testing cell
assert len(resultdf6) == 1
assert resultdf6.iloc[0,0] == "INX"

**Q7:** For every country in `indicators`, find the year in which the population hit its minimum value.  Select the country `code`, alias your new column as `min_year`, and also select the minimum population, aliased as `min_pop`.  You can have more than one row per country, if there is a tie.

_Hint:_ You should first find the `min_pop` for each country, then find all rows where `pop = min_pop`.  However, be careful that you do not match the `min_pop` of one country with the `pop` of another.

In [25]:
query = """SELECT ind.code, ind.year as min_year, minPop.min_pop FROM
(SELECT code, year , MIN(pop) as min_pop
FROM indicators
GROUP BY code) 
AS minPop INNER JOIN indicators AS ind 
ON (minPop.min_pop == ind.pop) 
AND (minPop.code == ind.code)
"""


resultset = %sql $query
resultdf7 = resultset.DataFrame()
print(len(resultdf7))
resultdf7.head()

 * sqlite:///../../dbfiles/book.db
   sqlite:///../../dbfiles/school.db
Done.
618


,code,min_year,min_pop
0,ABW,1960,0.05
1,AFG,1960,9.00
2,AGO,1960,5.45
3,ALB,1960,1.61
4,AND,1960,0.01


In [26]:
# Testing cell
assert resultdf7.shape == (618, 3)
assert list(resultdf7.iloc[0,:]) == ["ABW", 1960, 0.05]
assert list(resultdf7.iloc[617,:]) == ["TUV", 2018, 0.01]

**Q8:** For every country in `indicators`, find the _most recent_ year in which the population hit its minimum value.  Alias your new column as `min_year`, and also select the country code.

_Hint:_ Use your previous query as a subquery (so you'll have nested subqueries) and group by country code.

In [30]:
query = """SELECT code, MAX(min_year) AS min_year, min_pop FROM
(SELECT ind.code, ind.year as min_year, minPop.min_pop FROM
(SELECT code, year , MIN(pop) as min_pop
FROM indicators
GROUP BY code) 
AS minPop INNER JOIN indicators AS ind 
ON (minPop.min_pop == ind.pop) 
AND (minPop.code == ind.code))
GROUP BY code
"""


resultset = %sql $query
resultdf8 = resultset.DataFrame()
print(len(resultdf8))
resultdf8.head()

 * sqlite:///../../dbfiles/book.db
   sqlite:///../../dbfiles/school.db
Done.
217


,code,min_year,min_pop
0,ABW,1960,0.05
1,AFG,1960,9.00
2,AGO,1960,5.45
3,ALB,1960,1.61
4,AND,1961,0.01


In [31]:
# Testing cell
assert resultdf8.shape == (217, 3)
assert list(resultdf8.iloc[0,:]) == ["ABW", 1960, 0.05]
assert list(resultdf8.iloc[207,:]) == ["VGB", 1986, 0.01]

---

---

## Part C

**Q9:** How much time (in minutes/hours) did you spend on this homework assignment?

3 hours

**Q10:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone